In [1]:
import sys
import os
import logging
import sqlite3

from datetime import datetime

Create the logger

In [2]:
# Create a custom logger named 'db_logger'
logger = logging.getLogger("db_logger")
logger.setLevel(logging.INFO)  # Set the lowest level to INFO, so DEBUG logs are ignored

# Define log file path
log_file_path = "logs/db_connection.log"  # Log file inside 'logs' directory

# Ensure the directory for the log file exists, create it if it doesn't
log_dir = os.path.dirname(log_file_path)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Create handlers
file_handler = logging.FileHandler(log_file_path)
console_handler = logging.StreamHandler()

# Set level for handlers to INFO and above
file_handler.setLevel(logging.INFO)  # Log INFO and higher levels to the file
console_handler.setLevel(logging.INFO)  # Print INFO and higher levels to the console

# Create formatters and add them to handlers
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
file_handler.setFormatter(formatter)
console_handler.setFormatter(formatter)

# Add handlers to the logger
logger.addHandler(file_handler)
logger.addHandler(console_handler)

# Log a message to check if file handler works
logger.info("Logger initialized successfully")

2024-09-10 14:30:12,668 - db_logger - INFO - Logger initialized successfully


Define the data

In [3]:
# Define multiple rows of data
data = [
    "https://example.com/job_offer1",
    "https://example.com/job_offer2",
    "https://example.com/job_offer3",
    "https://example.com/job_offer4",
    "https://example.com/job_offer5",
]

Check if the table exists, if not - create it

In [4]:
try:
    # Connect to the SQLite database
    job_offers_db = sqlite3.connect("job_offers_db.db")
    cursor = job_offers_db.cursor()

    logger.info("Attempting to connect to the database")

    # Query sqlite_master to check if the "offers" table exists
    cursor.execute(
        """
        SELECT name
        FROM sqlite_master 
        WHERE type='table' AND name='offers';
        """
    )

    # Fetch the result
    result = cursor.fetchone()

    if not result:
        logger.info("The offers table doesn't exists - creating one")

        cursor.execute(
            """
            CREATE TABLE "offers" (
                "offer_id" INTEGER NOT NULL UNIQUE,
                "link" TEXT NOT NULL,
                "date_created" TEXT NOT NULL,
                "date_updated" TEXT NOT NULL,
                "active" INTEGER NOT NULL,
                PRIMARY KEY("offer_id" AUTOINCREMENT)
            )
            """
        )

        # Commit the transaction
        job_offers_db.commit()

    else:
        logger.info("The offers table already exists")

except Exception as e:
    logger.error(f"Error while connecting to a database: \n{type(e).__name__}: {e}")
    sys.exit(1)

else:
    # Log successful connection
    logger.info("Connected to the database successfully")

2024-09-10 14:30:12,692 - db_logger - INFO - Attempting to connect to the database
2024-09-10 14:30:12,694 - db_logger - INFO - The offers table already exists
2024-09-10 14:30:12,695 - db_logger - INFO - Connected to the database successfully


Insert to or update existing rows in a table

In [5]:
for link in data:
    # Check if data row already exists in the database by the link
    cursor.execute(
        """
        SELECT * 
        FROM offers
        WHERE link = ?
        """,
        (link,),
    )

    if cursor.fetchone():  # If the row exists update it, else create a new one
        try:
            logger.info(f"{link} - already in database - updating")

            cursor.execute(
                """
                UPDATE offers
                SET 
                    date_updated = ?
                """,
                (datetime.today().strftime("%d-%m-%y"),),
            )

            # Commit the transaction
            job_offers_db.commit()

        except Exception as e:
            logger.error(
                f"Error udpating a row: \nlink: {link}\n{type(e).__name__}: {e}"
            )
    else:
        try:
            logger.info(f"{link} - not in database - creating")

            cursor.execute(
                """
                INSERT INTO offers (link, date_created, date_updated, active)
                    VALUES (?, ?, ?, ?)
                """,
                (
                    link,
                    datetime.today().strftime("%d-%m-%y"),
                    datetime.today().strftime("%d-%m-%y"),
                    1,
                ),
            )

            # Commit the transaction
            job_offers_db.commit()

        except Exception as e:
            logger.error(
                f"Error creating a row: \nlink: {link}\n{type(e).__name__}: {e}"
            )

2024-09-10 14:30:12,709 - db_logger - INFO - https://example.com/job_offer1 - not in database - creating
2024-09-10 14:30:12,717 - db_logger - INFO - https://example.com/job_offer2 - not in database - creating
2024-09-10 14:30:12,726 - db_logger - INFO - https://example.com/job_offer3 - already in database - updating
2024-09-10 14:30:12,728 - db_logger - INFO - https://example.com/job_offer4 - already in database - updating
2024-09-10 14:30:12,729 - db_logger - INFO - https://example.com/job_offer5 - already in database - updating


Set the state of the offer to false if date_updated is not a current date

In [6]:
try:
    logger.info(f"Updating the state of all offers")
    
    cursor.execute(
    """
    UPDATE offers
    SET
        active = CASE WHEN date_updated = ? THEN 1 ELSE 0 END
    """,
        (datetime.today().strftime("%d-%m-%y"),),
    )
    
    logger.info(f"Offers state updated")
    
except Exception as e:
    logger.error(f"Error updating a state of an offer: \n{type(e).__name__}: {e}")

2024-09-10 14:30:12,740 - db_logger - INFO - Updating the state of all offers
2024-09-10 14:30:12,742 - db_logger - INFO - Offers state updated


In [7]:
# Close the cursor and connection
cursor.close()
job_offers_db.close()

logger.info(f"Database connection closed")

2024-09-10 14:30:12,754 - db_logger - INFO - Database connection closed
